# Exercice: Modélisation orientée requête (Cassandra)

<table><tr>
    <td><img src="image/esi-sba.png" width="100" height="100"></td>
    <td><img src="image/cassandra.png" width="100" height="100"></td>
    </tr></table>



### Dans ce notebook, nous allons travailler sur la modélisation orientée requête sous la BDD No-SQL Cassandra, en particulier nous traitons les points suivants :

* Dénormaliser une BDD relationnelle en partant du principe 1 table per query 
* Définir la partie Primary Key pour chaque requête
* Optimiser vos modèles en choisissant la bonne Partition Key & Clustering Column
* Valider vos modèles 
* Tester quelques requêtes particulières


#### installer cassandra-driver afin de se connecter au cluster cassandra
! pip install cassandra-driver
#### More documentation can be found here:  https://datastax.github.io/python-driver/

#### Import Apache Cassandra python package

In [1]:
import cassandra

### créer une connection au cluster cassandra

In [3]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) # si cassandra est installé localement avec le port par défaut 9042
    session = cluster.connect()
except Exception as e:
    print(e)

### créer maintenant un keyspace nommé "iot_esi"

In [4]:
try:
    
  ### add code here 
  session.execute("""
    CREATE KEYSPACE IF NOT EXISTS iot_esi2
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""") 

except Exception as e:
    print(e)

### connecter à votre Keyspace  iot_esi 


In [5]:
try:
    
    ### add code here 
    session.set_keyspace('iot_esi2')
    
except Exception as e:
    print(e)

### Supposons qu'on a un système IoT qui stocke les données capturées par les différents Sensors en utilisant un modèle RELATIONNEL, tel que est décrit dans le schéma ER ci-dessous:

<table><tr>
    <td><img src="image/ex1_IoT_ER_model.png" width="500" height="500"></td>
      </tr></table>

### imaginons  que nous souhaitons analyser ces données à  travers 3 requêtes analytiques:
* `Query1= retourner les informations(name, nbSensors) de tous les networks d'une région donnée`
* `Query2= retourner les informations (name,type,localisation, characteristics) de tous les Sensors d'un Network donné.`
* `Query3= retourner le AVG_measurements par sensor,  par année et par mois.  Ordonner le résultat par  année(desc) et par mois(asc)`

#### Créer les trois tables qui correspondent aux requêtes précédentes, en indiquant pour chacune:
* Primary Key
* Partition Key
* Clustering Column

In [6]:
try:
    
    ### Create the table that corresponds to the Query1
    query1 = "CREATE TABLE IF NOT EXISTS Networks "
    query1 = query1 + "(nameNt text, region text, nbSensors int, PRIMARY KEY (region,nameNt))"
    session.execute(query1)

    ### Create the table that corresponds to the Query2
    query2="CREATE TABLE IF NOT EXISTS Sensor_By_Network"
    query2 = query2 + "(nameNt text, nameS text, type text, latitude float, longitude float,characteristics MAP<text,text>, PRIMARY KEY (nameNT, nameS))"
    session.execute(query2)
    
    
    ### Create the table that corresponds to the Query3
    query3="CREATE TABLE IF NOT EXISTS AVG_measurements"
    query3 = query3 + "(nameS text,  annee int, mois int, AVG_measurements float,  "
    query3=query3+" PRIMARY KEY (nameS, annee, mois)) with clustering order by (annee desc, mois asc) ;"
    session.execute(query3) 
    
except Exception as e:
        print(e)

### Une fois vous créez les différentes tables, essayez de les peupler en se basant sur le contenu de la BDD relationnelle.

<table><tr>
    <td><img src="image/data_tables.png" width="600" height="500"></td></tr>
  
</table>

### Insertion des données dans Table1

In [7]:
query = "INSERT INTO networks (nameNt, region, nbSensors) "
query = query + " VALUES (%s,%s, %s)"

try:
    session.execute(query, ("esi-net","ouest",2))
    session.execute(query, ("udl-net","ouest",1))
    session.execute(query, ("alger-net","center",2))
except Exception as e:
        print(e)  

### Insertion des données dans Table2

In [23]:
query = "INSERT INTO Sensor_By_Network (nameNt , nameS , type , latitude, longitude,characteristics)"
query = query + " VALUES (%s, %s, %s, %s, %s,%s)"

try:
    session.execute(query, ("esi-net","temp-11","temp",31,-96,{"accuracy":"medium","sensitivity":"high"}))
    session.execute(query, ("esi-net","temp-12","temp",31,-97,{"accuracy":"high","sensitivity":"high"}))
    session.execute(query, ("udl-net","temp-21","temp",32,-97,{"accuracy":"low","sensitivity":"medium"}))
    session.execute(query, ("alger-net","temp-31","temp",44,-111,{"accuracy":"medium","sensitivity":"medium"}))
    session.execute(query, ("alger-net","temp-32","temp",45,-111,{"accuracy":"high","sensitivity":"low"}))
except Exception as e :
        print(e)     

### Insertion des données dans Table3

In [11]:
query = "INSERT INTO  AVG_measurements (nameS,  annee, mois , AVG_measurements)"
query = query + " VALUES (%s, %s, %s, %s)"

try:
    session.execute(query, ("temp-11",2021,10, 11))
    session.execute(query, ("temp-11",2021,12, 2))
    session.execute(query, ("temp-11",2022,10, 16))
    session.execute(query, ("temp-11",2022,12, 9))
    
    session.execute(query, ("temp-12",2021,10, 12))
    session.execute(query, ("temp-12",2021,12, 3))
    session.execute(query, ("temp-12",2022,10, 16.5))
    session.execute(query, ("temp-12",2022,12, 9.5))
    
    session.execute(query, ("temp-21",2021,10, 12.5))
    session.execute(query, ("temp-21",2021,12, 2.5))
    session.execute(query, ("temp-21",2022,10, 16))
    session.execute(query, ("temp-21",2022,12, 8.5))
    
    session.execute(query, ("temp-31",2021,10, 16))
    session.execute(query, ("temp-31",2021,12, 6.5))
    session.execute(query, ("temp-31",2022,10, 18))
    session.execute(query, ("temp-31",2022,12, 11))
    
    session.execute(query, ("temp-32",2021,10, 14))
    session.execute(query, ("temp-32",2021,12, 4.5))
    session.execute(query, ("temp-32",2022,10, 16.5))
    session.execute(query, ("temp-32",2022,12, 8.5))
    
except Exception as e :
        print(e)   

### Valider votre modèle en exécutant les requêtes suivantes

* Retourner le nombre de sensors dans la région ouest"

* Retourner les sensors qui sont placés dans le network "esi-net"

* Retourner la température maximale enregistrée par le sensor "temp-11" pendant l'année 2021

In [15]:
# query1: Retourner le nombre de sensors dans la région ouest
query = "select sum(nbsensors) from Networks where region='ouest'"

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)           

Row(system_sum_nbsensors=3)


In [27]:
# query2: Retourner les sensors qui sont placés dans le network "esi-net"
query = "select * from Sensor_By_Network where nameNt='esi-net' "

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)    

Row(nament='esi-net', names='temp-11', characteristics=OrderedMapSerializedKey([('accuracy', 'medium'), ('sensitivity', 'high')]), latitude=31.0, longitude=-96.0, type='temp')
Row(nament='esi-net', names='temp-12', characteristics=OrderedMapSerializedKey([('accuracy', 'high'), ('sensitivity', 'high')]), latitude=31.0, longitude=-97.0, type='temp')


In [33]:
# query3: Retourner la température maximale enregistrée par le sensor "temp-11" pendant l'année 2021
query = "select max(AVG_measurements)  from AVG_measurements where names='temp-11' and annee=2021"

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)         

Row(system_max_avg_measurements=11.0)


### Essayons d'autres requêtes 

* `Retourner la température maximale enregistrée par le sensor "temp-11"  pendant le  mois d'octobre de toutes les années` 

In [34]:
# query4
query = "select max(AVG_measurements)  from AVG_measurements where names='temp-11' and mois=10"



try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)      

Error from server: code=2200 [Invalid query] message="PRIMARY KEY column "mois" cannot be restricted as preceding column "annee" is not restricted"


### Proposer une solution pour optimiser cette requête

In [35]:
### make your solution 
## table creation
### Create the table that corresponds to the Query4
query4="CREATE TABLE IF NOT EXISTS AVG_measurements_by_month"
query4 = query4 + "(nameS text,  annee int, mois int, AVG_measurements float,  "
query4=query4+" PRIMARY KEY (nameS, mois, annee));"
session.execute(query4) 

## insert data
query = "INSERT INTO  AVG_measurements_by_month (nameS,  annee, mois , AVG_measurements)"
query = query + " VALUES (%s, %s, %s, %s)"

try:
    session.execute(query, ("temp-11",2021,10, 11))
    session.execute(query, ("temp-11",2021,12, 2))
    session.execute(query, ("temp-11",2022,10, 16))
    session.execute(query, ("temp-11",2022,12, 9))
    
    session.execute(query, ("temp-12",2021,10, 12))
    session.execute(query, ("temp-12",2021,12, 3))
    session.execute(query, ("temp-12",2022,10, 16.5))
    session.execute(query, ("temp-12",2022,12, 9.5))
    
    session.execute(query, ("temp-21",2021,10, 12.5))
    session.execute(query, ("temp-21",2021,12, 2.5))
    session.execute(query, ("temp-21",2022,10, 16))
    session.execute(query, ("temp-21",2022,12, 8.5))
    
    session.execute(query, ("temp-31",2021,10, 16))
    session.execute(query, ("temp-31",2021,12, 6.5))
    session.execute(query, ("temp-31",2022,10, 18))
    session.execute(query, ("temp-31",2022,12, 11))
    
    session.execute(query, ("temp-32",2021,10, 14))
    session.execute(query, ("temp-32",2021,12, 4.5))
    session.execute(query, ("temp-32",2022,10, 16.5))
    session.execute(query, ("temp-32",2022,12, 8.5))
    
except Exception as e :
        print(e)   

In [39]:
# query4
query = "select max(AVG_measurements)  from AVG_measurements_by_month where names='temp-11' and mois=10"



try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)  

Row(system_max_avg_measurements=16.0)


###  drop  tables. 

In [40]:
query = "drop table IF EXISTS  Networks"
rows = session.execute(query)

query = "drop table IF EXISTS Sensor_By_Network "
rows = session.execute(query)

query = "drop table IF EXISTS AVG_measurements  "
rows = session.execute(query)



query = "drop table IF EXISTS AVG_measurements_by_month "
rows = session.execute(query)

### close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()